# Cart 树和剪枝
由于找到的数据集为连续数据集, 并且数据较多, 这里只构建 Cart 树并基于 Cart 树进行剪枝

## Cart 树

---

1. 核心特征
    - 使用基尼指数用来划分数据集
    - 采取二分法构建树
1. 相对 ID3 和 C4.5 树的特征
    - 可以处理连续数据和缺失数据
    - 二分法构建树, 构建速度比较快
    - 拟合优度更佳
    - 不停二分离散特征, 这让已经被选择过的标签可以继续参与选择
1. 基尼指数(Gini Index)
    - 基尼指数表示样本的 "**不纯度**", 基尼指数越大, 样本越不纯
    - 整体基尼指数的计算, 公式为$$Gini(D)=1-\sum_{i=1}^{n} p(x_{i})^{2}$$
    其中 $p(x_{i})$ 为分类 $i$ 出现的次数
    - 条件基尼指数的计算$$Gini(D|A=a)=\frac{|D|}{|D_{1}|}Gini(D_{1})+\frac{|D|}{|D_{2}|}Gini(D_{2})$$
    其中 $D_1$ 和 $D_2$ 分别是当特征 $A=a$ 时对数据集的划分. 当特征 $A$ 为连续型时, 要进行连续变量离散化处理, 也就是说, 应该使用 $A\geq a$ (或 $A\leq a$) 和 $A<a$ (或 $A> a$) 来划分数据集.

## 剪枝

---

### 前剪枝(pre-pruning)策略
1. 设定深度阈值, 深度达到阈值后停止树的生长
1. 定义实例数阈值，当达到某个结点的实例个数小于该阈值时就可以停止树的生长
### 后剪枝(post-pruning)策略
比较常用的策略一共有 3 种, 分别是 REP, PEP, 和CCP. 本例仅介绍和使用 CCP.

### CCP(代价复杂度剪枝)介绍
该算法为子树 $T_t$ 定义了代价 (cost) 和复杂度 (complexity), 以及一个可由用户设置的衡量代价与复杂度之间关系的参数 $\alpha$. 其中, 代价指在剪枝过程中因子树 $T_t$ 被叶节点替代而增加的错分样本, 复杂度表示剪枝后子树 $T_t$ 减少的叶结点数, 表面误差增益率 $\alpha$ 表示剪掉某节点的左右子树后树的复杂度降低程度与代价间的关系, 定义一个非叶子节点上的 $\alpha$ 值为
$$
\alpha=\frac{R(t)-R(T_{t})}{|N_{1}|-1}
$$
其中:
- $|N_1|$ 为子树中的节点个数
- $R(t)$ 为结点 $t$ 的错误代价, 计算公式为 $R(t)=r(t)\cdot p(t)=m/n$, $r(t)$ 为结点 $t$ 的错分样本率, $p(t)$为落入结点 $t$ 的样本占所有样本的比例, $m$ 为节点 $t$ 的分类错误的样本数, $n$ 为总的样本数.
- $R(T_{t})$：子树 $T_t$ 错误代价，计算公式为 $R(T_{t})=\sum R(i)$, $i$ 为子树 $T_t$ 的叶节点.

对于完全决策树的每个非叶子节点, 计算它们的 $\alpha$ 值, 然后循环剪掉 $\alpha$ 最小的子树, 直到只剩下根节点. 每一步都可得到一个剪枝树, 由此得到一个剪枝树序列 $\{T_{0},\ T_{1},\ ...,\ T{m}\}$, 其中 $T_0$ 是完全决策树, $T_m$ 是根节点. 

然后使用 1-SE(1 standard error of minimum error) 规则从上述序列中挑选一个最优子树. 

1-SE 规则: 假定一个验证样本集含有 $N'$ 个样本, 使用使用上述剪枝树对样本集分类, 记 $T_i$ 上被错分的样本数为 $E_i$ 并令 $E'$ 为序列 $\{E_i \}$ 的最小值. 计算标准样本误
$$
SE(E')=\sqrt{\frac{E'(N'-E')}{N'}}
$$

找到满足 $E_i\leq E'+SE(E')$ 且节点数最少的剪枝树即为所求最优剪枝树 $T_i$.

## 程序实现

---

### 创建数据
BaseData中第一行为标签数据, 前3/4为样本集, 后1/4为验证集, 当satisfaction_level大于等于0.75时为满意, 否则为不满意

In [1]:
from pandas import read_csv as read
def DataCreate():
    BaseData=read('./assets/data.csv',header=0,converters={'happy':lambda x:bool(int(x))})
    #从 data.csv 读取数据, 第一列为索引列, 第一行为标题行, converters 表示对 satisfaction_level 列做改动
    length=len(BaseData)
    Dataset=BaseData.iloc[0:3*length//4] #取前3/4为样本集
    Varifyset=BaseData.iloc[3*length//4:length] #取后1/4为验证集
    return Dataset, Varifyset

### 计算基尼指数
这里计算的是非条件的基尼指数, 条件基尼指数的计算只要切分数据集后用非条件的基尼指数进行四则运算即可

In [2]:
def Gini(Dataset):
    KindIndex=Dataset.columns[-1] #取最后一列的列标题, 也就是特征的标签
    KindNum=dict.fromkeys(set(Dataset[KindIndex].tolist()),0) #取最后一列的数据,并去除重复转换为字典
    for index, row in Dataset.iterrows(): #iterrows方法, 返回一个生成器, 第一项为数据索引, 第二项为数据
        KindNum[row[-1]] += 1 #计算每种分类的数量
    GiniIndex = 1 #计算sum(p(xi)**2)
    for example in KindNum.keys():
        GiniIndex -= (KindNum[example]/len(Dataset))**2
    return GiniIndex #返回基尼指数

### 数据切分
即求公式中的 $D_1$ 和 $D_2$, 切分点为 label 中的 value 值. 

只认为 string 是非连续的, 其它类型的切分都使用数值切分法

In [3]:
def SetSpilt(Dataset, label, value):
    if type(value) == str : #如果类型属于字符串,则执行离散切分
        subset1 = Dataset[Dataset[label] == value] #使用了dataframe的布尔索引功能, 筛选 label 列中等于 value 的数据, 下同
        subset2 = Dataset[Dataset[label] != value]
    else: #如果不是字符串, 则执行离散化切分
        subset1=Dataset[Dataset[label] >= value]
        subset2=Dataset[Dataset[label] < value]
    return subset1, subset2 #返回值为切分后的两个数据集
# Dataset, varifyset=DataCreate()
# set1,set2=SetSpilt(Dataset,'package','a')

### 寻找最佳切分点
- 比较所有标签切分点的基尼指数, 然后取基尼指数最小的特征进行切分
- 对于连续数据切分点的选择, 把可能出现的取值排序, 然后取中位数, 利用中位数切分, 这就是连续数据离散化的思想

In [4]:
def BestSpilt(Dataset):
    bestGini = 1 #初始化最优Gini指数为1
    modifier = 1 / len(Dataset) #计算1/|D|
    BestSpiltPoint=() #最佳切分点, 是一个元组
    for label in Dataset.columns[:-2]: #从所有特征中循环
        FeatureSet = set(Dataset[label].tolist()) #特征值的集合
        if (len(FeatureSet) == 1): #如果该特征下只有一个特征值, 则已经被使用过, 跳过
            continue
        for feature in FeatureSet:
            subSet1,subSet2 = SetSpilt(Dataset, label, feature) #划分数据集
            if (len(subSet1) == 0)or(len(subSet2) == 0):
                continue
            conditionGini = modifier * (Gini(subSet1)/len(subSet1) + Gini(subSet2)/len(subSet2)) #计算条件gini指数
            if conditionGini < bestGini: #寻找最小gini指数
                bestGini = conditionGini
                BestSpiltPoint=((label,feature),subSet1,subSet2) #更新返回值
    if len(BestSpiltPoint)==0: #当前样本集在所有特征上的所有特征值都相同
        return 0
    return BestSpiltPoint

### 寻找数据集中最多的分类

In [5]:
def mostFeature(Dataset):
    KindIndex=Dataset.columns[-1]
    KindNum=dict.fromkeys(set(Dataset[KindIndex].tolist()),0) #取最后一列的数据,并去除重复转换为字典
    for index, row in Dataset.iterrows(): #iterrows方法, 返回一个生成器, 第一项为数据索引, 第二项为数据, 可用于循环
        KindNum[row[-1]] += 1 #计算每种分类的数量
    return max(KindNum,key=KindNum.get) #返回最大值

### 创建树的过程
本环节已经包含前剪枝

In [13]:
def CreateMytree(Dataset, depth=0):
    depth += 1
    #递归出口1, 如果决策树的深度大于8层, 则停止递归,返回数据集中最多的特征
    if depth >= 8:
        return [mostFeature(Dataset), 0, 0]
    thrval = 0.05
    leastNum = 5
    #递归出口2, 如果当前节点包含的样本数小于leastNum, 或gini指数小于阈值thrval, 终止递归, 返回数据集中最多的特征
    if (len(Dataset)<leastNum) or (Gini(Dataset) < thrval):
        return [mostFeature(Dataset), 0, 0]
    #递归出口3, 如果当前节点包含样本在所有特征上的特征值都相同, 而样本仍然没有分纯(因为通过了递归出口2)则终止递归, 返回数据集中最多的特征
    if BestSpilt(Dataset) == 0:
        return [mostFeature(Dataset), 0, 0]
    ((label,feature),subSet1,subSet2)=BestSpilt(Dataset)
    myTree={'count':[0, 0], label:{}} #为了计算alpha值, 增加了一个用于计数的列表, 列表第一项是落入该节点的数据总数, 第二项是没有被正确分类的数据总数
    myTree[label][feature]=CreateMytree(subSet1, depth)
    myTree[label]['others']=CreateMytree(subSet2,depth)
    return myTree

### 计算树中的叶节点个数 $|N_1|$

In [7]:
def CountLeaf(Tree):
    sum = 0
    #递归计算树的叶节点, 如果子节点还不是叶节点, 则计算子树的叶节点个数, 如果是叶节点, 则返回
    for value in Tree.values():
        if type(value) == dict:
            sum += CountLeaf(value)
        elif type(value) == list: #防止计数列表的干扰
            continue
        else:
            sum += 1
    return sum

### 利用所得决策树对单条样本进行分类

In [18]:
def SinglePredict(example, Tree):
    tree = Tree
    path=[]
    while True:
        try:
        #如果递归应当结束, 则tree将不再是字典, keys方法将抛出错误, 表明递归结束
            label = list(tree.keys())[1]
            path.append(tree)
        except:
        #路径的计数列表第一项加一, 且如果该数据没有被正确分类, 则计数列表的第二项加一
            tree[1] += 1
            for value in path:
                value['count'][0] += 1
            if example.values[-1] != tree[0]:
                tree[2] += 1
                for value in path:
                    value['count'][1] += 1
            return Tree
        value = list(tree[label].keys())[0]
        if type(value) == str:
            if example[label] == value: #如果相等则落入左子树, 不相等则落入右子树
                tree = tree[label][value]
            else:
                tree = tree[label]['others']
        else:
            if example[label] >= value:
                tree = tree[label][value]
            else:
                tree = tree[label]['others']

### 对于所有数据进行预测并形成完全剪枝树 $T_0$

In [19]:
def Predict(varifyset, Tree):
    for index, row in varifyset.iterrows():
        SinglePredict(row, Tree)
    return Tree
dateset, varifyset = DataCreate()
tree = CreateMytree(dateset)
Predict(varifyset, tree)

{'count': [36, 15],
 'infoavail': {3: {'count': [36, 15],
   'policetrust': {2: {'count': [34, 14],
     'streetquality': {2: {'count': [32, 13],
       'infoavail': {5: {'count': [20, 9],
         'schoolquality': {2: {'count': [20, 9],
           'policetrust': {5: {'count': [2, 1],
             'schoolquality': {3: [True, 1, 0], 'others': [True, 1, 1]}},
            'others': {'count': [18, 8],
             'schoolquality': {5: [False, 3, 2], 'others': [True, 15, 6]}}}},
          'others': [False, 0, 0]}},
        'others': {'count': [12, 4],
         'policetrust': {3: {'count': [11, 4],
           'schoolquality': {5: [False, 0, 0],
            'others': {'count': [11, 4],
             'streetquality': {5: [True, 1, 1], 'others': [False, 10, 3]}}}},
          'others': [False, 1, 0]}}}},
      'others': [False, 2, 1]}},
    'others': [True, 2, 1]}},
  'others': [False, 0, 0]}}

### 计算表面误差增益率
计算的是树的根节点的 $\alpha$, 它的计算是由上自下的

In [ ]:
def CountR(Tree, leng):
    sum = 0
    #递归计算树的叶节点, 如果子节点还不是叶节点, 则计算子树的叶节点, 如果是叶节点, 则返回
    for value in Tree.values():
        if type(value) == dict:
            sum += CountR(value)
        elif (type(value) == list) and len(value) == 2: #防止计数列表的干扰
            continue
        elif (type(value) == list) and len(value) == 3:
            sum += value[2] / leng
    return sum

def alpha(Tree, leng):
    RT = Tree['count'][1] / Tree['count'][0]
    N = CountLeaf(Tree)
    r = CountR(Tree, leng)
    return (RT - r) / (N - 1)

### 进行后剪枝
每次找到 $\alpha$ 值最小的内部节点, 对它进行剪枝, 令其等于分到此处的样本的最多的那一类, 返回值为剪枝序列.

In [10]:
def postPruning(Tree):
    PrunedTreeArray = [Tree]
    alphamin = 1.0
    
    return PrunedTreeArray